In [7]:
%load_ext autoreload
%autoreload 2

In [3]:

import sys
sys.path.append('../src')
from textwrap import dedent
import pandas as pd 
from glob import glob 
import re 
import uuid
import chromadb

from llm_utils import chat_completion_openai
from configs import llm_scenarios_config, models_config


# Create table with embs

In [25]:
nsfw_df = pd.DataFrame()

nudes = glob('../images/angel/**/*.png', recursive=True)
nsfw_df['img_path'] = nudes
nsfw_df['prompt'] = nsfw_df['img_path'].apply(lambda x: x.split('/')[-1].split('.')[0])
nsfw_df['tag_nsfw'] = 1
nsfw_df['tag'] = nsfw_df['img_path'].apply(lambda x: x.split('/')[-2])
nsfw_df['tag'] = nsfw_df['tag'].replace('angel', None)
nsfw_df

,img_path,prompt,tag_nsfw,tag
0,"../images/angel/laying on the bed, legs spread...","laying on the bed, legs spread, showing pussy",1,None
1,"../images/angel/sitting on a gaming chair, pus...","sitting on a gaming chair, pussy, naked",1,None
2,"../images/angel/sitting on a floor, wearing bl...","sitting on a floor, wearing black shoes, black...",1,None
3,"../images/angel/having sex, black dick, fuckin...","having sex, black dick, fucking, beeing fucked...",1,None
4,"../images/angel/dildo, vibrator, in a car, pus...","dildo, vibrator, in a car, pussy, masturbate",1,None
...,...,...,...,...
98,../images/angel/nudes/nude selfie/selfie21.png,selfie21,1,nude selfie
99,../images/angel/nudes/nude selfie/selfie23.png,selfie23,1,nude selfie
100,../images/angel/nudes/nude selfie/selfie22.png,selfie22,1,nude selfie
101,../images/angel/nudes/nude selfie/selfie9.png,selfie9,1,nude selfie


In [26]:
ysfw_df = pd.DataFrame()

nudes = glob('../images/dressed/**/*.png', recursive=True)
ysfw_df['img_path'] = nudes
ysfw_df['prompt'] = ysfw_df['img_path'].apply(lambda x: x.split('/')[-1].split('.')[0])
ysfw_df['tag_nsfw'] = 0
ysfw_df['tag'] = ysfw_df['img_path'].apply(lambda x: x.split('/')[-2])
ysfw_df['tag'] = ysfw_df['tag'].replace('angel', None)
ysfw_df

,img_path,prompt,tag_nsfw,tag
0,../images/dressed/hobbies/school_uniform_books...,school_uniform_books,0,hobbies
1,../images/dressed/hobbies/home_books1.png,home_books1,0,hobbies
2,../images/dressed/hobbies/hiking_jacket1.png,hiking_jacket1,0,hobbies
3,../images/dressed/hobbies/hiking_jacket.png,hiking_jacket,0,hobbies
4,../images/dressed/hobbies/home_videogames2.png,home_videogames2,0,hobbies
5,../images/dressed/hobbies/morning_run.png,morning_run,0,hobbies
6,../images/dressed/hobbies/home_videogames1.png,home_videogames1,0,hobbies
7,../images/dressed/hobbies/basketball.png,basketball,0,hobbies
8,../images/dressed/hobbies/morning_run1.png,morning_run1,0,hobbies
9,../images/dressed/hobbies/basketball1.png,basketball1,0,hobbies


In [27]:
df = pd.concat([ysfw_df, nsfw_df])
df['prompt'] = df['prompt'].str.replace('_', ', ')
df['prompt'] = df['prompt'].apply(lambda x: re.sub(r'\d', '', x))

def add_prefix(row):
    prefix = "tag_nsfw" if row['tag_nsfw'] else "sfw"
    return f"{prefix}, {row['prompt']}"

df['prompt'] = df.apply(add_prefix, axis=1)
df['prompt'] = df['prompt'] + ', ' + df['tag'].astype(str)
df

,img_path,prompt,tag_nsfw,tag
0,../images/dressed/hobbies/school_uniform_books...,"sfw, school, uniform, books, hobbies",0,hobbies
1,../images/dressed/hobbies/home_books1.png,"sfw, home, books, hobbies",0,hobbies
2,../images/dressed/hobbies/hiking_jacket1.png,"sfw, hiking, jacket, hobbies",0,hobbies
3,../images/dressed/hobbies/hiking_jacket.png,"sfw, hiking, jacket, hobbies",0,hobbies
4,../images/dressed/hobbies/home_videogames2.png,"sfw, home, videogames, hobbies",0,hobbies
...,...,...,...,...
98,../images/angel/nudes/nude selfie/selfie21.png,"tag_nsfw, selfie, nude selfie",1,nude selfie
99,../images/angel/nudes/nude selfie/selfie23.png,"tag_nsfw, selfie, nude selfie",1,nude selfie
100,../images/angel/nudes/nude selfie/selfie22.png,"tag_nsfw, selfie, nude selfie",1,nude selfie
101,../images/angel/nudes/nude selfie/selfie9.png,"tag_nsfw, selfie, nude selfie",1,nude selfie


In [31]:
df['img_uuid'] = df['img_path'].apply(lambda x: uuid.uuid4()).astype(str)

one_hot = pd.get_dummies(df['tag'], prefix='tag')
df = pd.concat([df, one_hot], axis=1)
df[:2]

,img_path,prompt,tag_nsfw,tag,img_uuid,tag_costume-lingerie,tag_feet,tag_hobbies,tag_indoors,tag_nude selfie,tag_outdoors,tag_squirt,embs
0,../images/dressed/hobbies/school_uniform_books...,"sfw, school, uniform, books, hobbies",0,hobbies,36d1b9f1-ba54-46a9-bc29-10122b0ee209,0,0,1,0,0,0,0,"[0.019440174102783203, -0.027745665982365608, ..."
1,../images/dressed/hobbies/home_books1.png,"sfw, home, books, hobbies",0,hobbies,bfba038d-5617-4ff9-bef2-9679cc868a3b,0,0,1,0,0,0,0,"[-0.022166674956679344, -0.026834778487682343,..."


In [32]:
df.reset_index().to_feather('../chroma-data/angel.feather')

In [8]:
len(df.iloc[0].to_dict()['embs'])

1024

# Add to chroma

In [4]:
persona_name = 'angel'

df = pd.read_feather(f'../images/{persona_name}.feather').reset_index(drop=True)
df[:1]

,index,img_path,prompt,tag_nsfw,tag,img_uuid,tag_costume-lingerie,tag_feet,tag_hobbies,tag_indoors,tag_nude selfie,tag_outdoors,tag_squirt,embs
0,0,../images/dressed/hobbies/school_uniform_books...,"sfw, school, uniform, books, hobbies",0,hobbies,36d1b9f1-ba54-46a9-bc29-10122b0ee209,0,0,1,0,0,0,0,"[0.019440174102783203, -0.027745665982365608, ..."


In [21]:
df.columns

Index(['index', 'img_path', 'prompt', 'tag_nsfw', 'tag', 'img_uuid',
       'tag_costume-lingerie', 'tag_feet', 'tag_hobbies', 'tag_indoors',
       'tag_nude selfie', 'tag_outdoors', 'tag_squirt', 'embs'],
      dtype='object')

In [8]:
# Connect to Chroma client
chroma_client = chromadb.HttpClient(host="localhost", port=8000, settings=chromadb.Settings(allow_reset=True, anonymized_telemetry=False))
collection = chroma_client.get_or_create_collection(name=persona_name, metadata={"hnsw:space": "cosine"} )

_client.py          :1026 2024-08-07 07:45:08,776 HTTP Request: GET http://localhost:8000/api/v1/tenants/default_tenant "HTTP/1.1 200 OK"
_client.py          :1026 2024-08-07 07:45:08,785 HTTP Request: GET http://localhost:8000/api/v1/databases/default_database?tenant=default_tenant "HTTP/1.1 200 OK"
_client.py          :1026 2024-08-07 07:45:08,796 HTTP Request: POST http://localhost:8000/api/v1/collections?tenant=default_tenant&database=default_database "HTTP/1.1 200 OK"


In [9]:
metadata_cols = [x for x in df.columns if x.startswith('tag_')] + ['prompt', 'img_uuid']
print('metadata_cols:', metadata_cols)
metadatas = df[metadata_cols].to_dict(orient='records')
ids = df['img_uuid'].astype(str).tolist()
documents = df['img_path'].tolist()
collection.add(documents=documents, embeddings=[x.tolist() for x in df['embs'].tolist()], metadatas=metadatas, ids=ids)

print(f"Изображения добавлены в {persona_name}")

_client.py          :1026 2024-08-07 07:45:08,919 HTTP Request: GET http://localhost:8000/api/v1/pre-flight-checks "HTTP/1.1 200 OK"
_client.py          :1026 2024-08-07 07:45:09,092 HTTP Request: POST http://localhost:8000/api/v1/collections/b8889533-c023-4ad7-b1b0-6266a9246d96/add "HTTP/1.1 201 Created"


metadata_cols: ['tag_nsfw', 'tag_costume-lingerie', 'tag_feet', 'tag_hobbies', 'tag_indoors', 'tag_nude selfie', 'tag_outdoors', 'tag_squirt', 'prompt', 'img_uuid']
Изображения добавлены в angel


# Querie examples

In [14]:
# Get all data
all_data = collection.get()
print("keys:", all_data.keys())
print("Number of rows:", len(all_data['ids']))


_client.py          :1026 2024-08-07 07:46:54,069 HTTP Request: POST http://localhost:8000/api/v1/collections/b8889533-c023-4ad7-b1b0-6266a9246d96/get "HTTP/1.1 200 OK"


keys: dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'data', 'uris', 'included'])
Number of rows: 157
